In [1]:
cd /mnt/data

/mnt/data


In [6]:
tenx_channel = '10X_P4_3'

In [7]:
! aws s3 sync s3://czbiohub-maca/10x_data/$tenx_channel/ $tenx_channel/

In [8]:
ls -lha /mnt/data/$tenx_channel/

total 27G
drwxrwxr-x  2 ubuntu ubuntu 4.0K Sep 13 23:26 ./
drwxr-xr-x 25 ubuntu root   4.0K Sep 21 01:30 ../
-rw-rw-r--  1 ubuntu ubuntu  15M Aug 22  2017 10X_P4_3.mus.cell-gene.csv
-rw-rw-r--  1 ubuntu ubuntu  14G Aug 22  2017 10X_P4_3.tgz
-rw-rw-r--  1 ubuntu ubuntu 2.8K Aug 31  2017 barcodes.tsv
-rw-rw-r--  1 ubuntu ubuntu 340K Aug 31  2017 genes.tsv
-rw-rw-r--  1 ubuntu ubuntu 5.4M Aug 31  2017 matrix.mtx
-rw-rw-r--  1 ubuntu ubuntu  615 Aug 22  2017 metrics_summary.csv
-rw-rw-r--  1 ubuntu ubuntu  14G Jun 19 02:34 possorted_genome_bam.bam
-rw-rw-r--  1 ubuntu ubuntu 4.7M Jun 19 02:34 possorted_genome_bam.bam.bai
-rw-rw-r--  1 ubuntu ubuntu 7.4M Sep 15  2017 raw_gene_bc_matrices_h5.h5
-rw-rw-r--  1 ubuntu ubuntu 2.6M Aug 22  2017 web_summary.html


In [9]:
ksizes = 21, 27, 33, 51
protein = True
dna = True
seed = 42
track_abundance = True
scaled = 1000
num_hashes = 0
input_is_protein = False 
check_sequence = False

In [10]:
import pandas as pd
barcodes = set(pd.read_csv(f'/mnt/data/{tenx_channel}/barcodes.tsv', squeeze=True))
genes = set(pd.read_csv(f'/mnt/data/{tenx_channel}/genes.tsv', squeeze=True))
len(barcodes)

/home/ubuntu/anaconda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/ubuntu/anaconda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


148

In [13]:
from sourmash import DEFAULT_SEED, MinHash, load_sbt_index, create_sbt_index
from sourmash import signature as sig
from sourmash import sourmash_args
from sourmash.logging import notify, error, print_results, set_quiet
from sourmash.sbtmh import SearchMinHashesFindBest, SigLeaf

from sourmash.sourmash_args import DEFAULT_LOAD_K
DEFAULT_COMPUTE_K = '21,31,51'

DEFAULT_N = 500
WATERMARK_SIZE = 10000
 
def make_minhashes():
    # one minhash for each ksize
    Elist = []
    for k in ksizes:
        if protein:
            E = MinHash(ksize=k, n=num_hashes,
                        is_protein=True,
                        track_abundance=track_abundance,
                        scaled=scaled,
                        seed=seed)
            Elist.append(E)
        if dna:
            E = MinHash(ksize=k, n=num_hashes,
                        is_protein=False,
                        track_abundance=track_abundance,
                        scaled=scaled,
                        seed=seed)
            Elist.append(E)
    return Elist

def add_seq(Elist, seq, input_is_protein, check_sequence):
    for E in Elist:
        if input_is_protein:
            E.add_protein(seq)
        else:
            E.add_sequence(seq, not check_sequence)

def build_siglist(Elist, filename, name=None):
    return [ sig.SourmashSignature(E, filename=filename,
                                   name=name) for E in Elist ]

def save_siglist(siglist, output_fp, filename=None):
    # save!
    if output_fp:
        sig.save_signatures(siglist, args.output)
    else:
        if filename is None:
            raise Exception("internal error, filename is None")
        with open(filename, 'w') as fp:
            sig.save_signatures(siglist, fp)
    notify('saved {} signature(s). Note: signature license is CC0.'.format(len(siglist)))

In [14]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [15]:
import pandas as pd

annotations = pd.read_csv('~/tabula-muris/00_data_ingest/18_global_annotation_csv/annotations_droplet.csv')
annotations.columns = annotations.columns.map(lambda x: x.replace('.', '_'))

annotations_subset = annotations.loc[annotations.cell.str.startswith(tenx_channel)]
annotations_subset['barcode'] = annotations_subset.cell.str.split('_').str[-1] + '-1'
annotations_subset['name'] = annotations_subset.apply(
    lambda x: 'cell_ontology_class:{cell_ontology_class}|tissue:{tissue}|free_annotation:{free_annotation}|mouse_id:{mouse_id}|{cell}'.format(**x), axis=1)
annotations_subset['name'] = annotations_subset['name'].str.replace(" ", "_")
annotations_subset = annotations_subset.set_index('barcode')
print(annotations_subset.shape)
annotations_subset.head()

/home/ubuntu/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (10,12,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/ubuntu/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/home/ubuntu/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


(149, 21)


/home/ubuntu/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


,cell,cell_ontology_class,cell_ontology_id,channel,cluster_ids,free_annotation,mouse_id,mouse_sex,subsetA,subsetA_cluster_ids,...,subsetB_cluster_ids,subsetC,subsetC_cluster_ids,subsetD,subsetD_cluster_ids,subtissue,tissue,tissue_tSNE_1,tissue_tSNE_2,name
barcode,,,,,,,,,,,,,,,,,,,,,
AAAGTAGAGATGCCAG-1,10X_P4_3_AAAGTAGAGATGCCAG,bladder cell,CL:1001319,10X_P4_3,0,Bladder mesenchymal cell,3-M-8,M,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Bladder,-28.034252,-13.407488,cell_ontology_class:bladder_cell|tissue:Bladde...
AACCGCGTCCAACCAA-1,10X_P4_3_AACCGCGTCCAACCAA,bladder cell,CL:1001319,10X_P4_3,2,Bladder mesenchymal cell,3-M-8,M,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Bladder,-15.890827,7.453387,cell_ontology_class:bladder_cell|tissue:Bladde...
AACTCCCGTCGGGTCT-1,10X_P4_3_AACTCCCGTCGGGTCT,bladder cell,CL:1001319,10X_P4_3,5,Bladder mesenchymal cell,3-M-8,M,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Bladder,-11.131739,8.426802,cell_ontology_class:bladder_cell|tissue:Bladde...
AACTCTTAGTTGCAGG-1,10X_P4_3_AACTCTTAGTTGCAGG,bladder urothelial cell,CL:1001428,10X_P4_3,3,Luminal bladder epithelial cell,3-M-8,M,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Bladder,21.559201,6.817286,cell_ontology_class:bladder_urothelial_cell|ti...
AACTCTTTCATAACCG-1,10X_P4_3_AACTCTTTCATAACCG,bladder cell,CL:1001319,10X_P4_3,2,Bladder mesenchymal cell,3-M-8,M,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Bladder,-5.939946,19.630456,cell_ontology_class:bladder_cell|tissue:Bladde...


In [16]:
def maybe_get_name(barcode):
    try:
        return names[barcode]
    except KeyError:
        return barcode

In [17]:

import pysam
from tqdm import tqdm
import itertools


# bam_filename = '/mnt/data/10X_P7_8/lung_ptprc.bam'

def make_10x_signatures(bam_filename):
    output = bam_filename.replace('.bam', '.sig')

    bam_file = pysam.AlignmentFile(bam_filename, mode='rb')
    cell_seqs = {barcode: make_minhashes() for barcode in barcodes}

    for a in tqdm(bam_file):
        if (a.mapq == 255                                    # high quality mapping
            and a.has_tag('CB') and a.get_tag('CB') in barcodes  # in our set of barcodes,
    #         and a.has_tag('GN') and a.get_tag['GN'] in genes   # that maps to a single gene,
    #         and a.has_tag('RE') and a.get_tag('RE') == 'E'   # specifically to an exon,
            and a.has_tag('UB')):                            # and has a good UMI

            barcode = a.get_tag('CB')
    #         print(a)
            # if this isn't marked a duplicate, count it as a UMI
            if not a.is_duplicate:
    #             print(f"Adding {a.seq} to {barcode}")
                add_seq(cell_seqs[barcode], a.seq,
                                input_is_protein, check_sequence)
    cell_signatures = [build_siglist(seqs, filename=bam_filename, name=maybe_get_name(barcode)) 
                       for barcode, seqs in cell_seqs.items()]
    signatures_flat = list(itertools.chain(*cell_signatures))
    save_siglist(signatures_flat, output_fp=False, filename=output)

# %lprun -f make_10x_signatures make_10x_signatures(bam_filename)

In [19]:
import sourmash

In [20]:
sourmash.__file__

'/home/ubuntu/anaconda/lib/python3.6/site-packages/sourmash/__init__.py'

In [18]:
make_10x_signatures(f'/mnt/data/{tenx_channel}/possorted_genome_bam.bam')

900782it [00:26, 33844.03it/s]

ValueError: basic_string::substr: __pos (which is 15) > this->size() (which is 14)

900782it [00:40, 22518.21it/s]

In [21]:
%debug

> /mnt/data/sourmash/_minhash.pyx(178)sourmash._minhash.MinHash.add_sequence()

ipdb> dir()
['MinHash', '__builtins__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', '__test__', 'copy', 'dotproduct', 'get_max_hash_for_scaled', 'get_minhash_default_seed', 'get_minhash_max_hash', 'get_scaled_for_max_hash', 'hash_murmur', 'math']
ipdb> seq
*** NameError: name 'seq' is not defined
ipdb> u
> <ipython-input-13-39537ff44979>(38)add_seq()
     36             E.add_protein(seq)
     37         else:
---> 38             E.add_sequence(seq, not check_sequence)
     39 
     40 def build_siglist(Elist, filename, name=None):

ipdb> print(seq)
NGGCCCAAGGAGAAGAGCGTGGCCGGCAAGGTGTGCNTCATCACGGGCNCNGNCNGCGNCNTNGNCNNNNTCTTNNCNNNNGNNNTNNCN
ipdb> qq
*** NameError: name 'qq' is not defined
ipdb> q


In [43]:
ls -lha /mnt/data/$tenx_channel/

total 37G
drwxrwxr-x  2 ubuntu ubuntu 4.0K Sep  8 01:31 ./
drwxr-xr-x 12 ubuntu root   4.0K Sep 10 19:03 ../
-rw-rw-r--  1 ubuntu ubuntu  62M Aug 31  2017 10X_P7_8.mus.cell-gene.csv
-rw-rw-r--  1 ubuntu ubuntu  19G Aug 31  2017 10X_P7_8.tgz
-rw-rw-r--  1 ubuntu ubuntu  12K Sep  1  2017 barcodes.tsv
-rw-rw-r--  1 ubuntu ubuntu 340K Sep  1  2017 genes.tsv
-rw-rw-r--  1 ubuntu ubuntu 3.6M Sep  7 21:58 lung_ptprc.bam
-rw-rw-r--  1 ubuntu ubuntu  69K Sep  7 21:58 lung_ptprc.bam.bai
-rw-rw-r--  1 ubuntu ubuntu 953K Sep 10 20:50 lung_ptprc.sig
-rw-rw-r--  1 ubuntu ubuntu  14M Sep  1  2017 matrix.mtx
-rw-rw-r--  1 ubuntu ubuntu  612 Aug 31  2017 metrics_summary.csv
-rw-rw-r--  1 ubuntu ubuntu  19G Jun 19 06:16 possorted_genome_bam.bam
-rw-rw-r--  1 ubuntu ubuntu 5.5M Jun 19 06:16 possorted_genome_bam.bam.bai
-rw-rw-r--  1 ubuntu ubuntu 193M Sep 11 00:22 possorted_genome_bam.sig
-rw-rw-r--  1 ubuntu ubuntu 9.2M Sep 16  2017 raw_gene_bc_matrices_h5.h5
-rw-rw-r--  1 ubuntu ubuntu 2.6M Aug 31  201

In [19]:
ls -lha /mnt/data/sourmash_databases/

total 13M
drwxrwxr-x  3 ubuntu ubuntu 4.0K Sep  8 18:05 ./
drwxr-xr-x 12 ubuntu root   4.0K Sep 10 19:03 ../
drwxrwxr-x  2 ubuntu ubuntu 3.4M Sep  8 18:05 .sbt.tabula-muris-dna-k21/
-rw-rw-r--  1 ubuntu ubuntu 8.8M Sep  8 18:05 tabula-muris-dna-k21.sbt.json


In [22]:
! sourmash categorize -h

usage: sourmash [-h] [-q] [-k KSIZE] [--threshold THRESHOLD]
                [--traverse-directory] [--protein] [--no-protein] [--dna]
                [--no-dna] [--csv CSV] [--load-csv LOAD_CSV]
                sbt_name queries [queries ...]

positional arguments:
  sbt_name              name of SBT to load
  queries               list of signatures to categorize

optional arguments:
  -h, --help            show this help message and exit
  -q, --quiet           suppress non-error output
  -k KSIZE, --ksize KSIZE
  --threshold THRESHOLD
  --traverse-directory
  --protein             choose a protein signature (default: False)
  --no-protein          do not choose a protein signature
  --dna                 choose a DNA signature (default: True)
  --no-dna              do not choose a DNA signature
  --csv CSV
  --load-csv LOAD_CSV


In [46]:
%%bash

sourmash search /mnt/data/maca-facs-sourmash_compute_all/A1-B000610-3_56_F-1-1.sig \
    /mnt/data/sourmash_databases/tabula-muris-dna-k21.sbt.json

When loading query from "/mnt/data/maca-facs-sourmash_compute_all/A1-B000610-3_56_F-1-1.sig"
8 signatures matching ksize and molecule type;
need exactly one. Specify --ksize or --dna/--protein.


In [ ]:
%%bash
sourmash categorize --ksize 21 --dna \
    /mnt/data/sourmash_databases/tabula-muris-dna-k21.sbt.json \
    /mnt/data/10X_P7_8/possorted_genome_bam.sig | head

```
 Wed 12 Sep - 20:57  ~/kmer-hashing   origin ☊ olgabot/index-tabula-muris 4● 
  sourmash categorize --ksize 21 --dna \
    /mnt/data/sourmash_databases/tabula-muris-dna-k21.sbt.json \
    /mnt/data/10X_P7_8/possorted_genome_bam.sig | head
found 1 files to query

...sig loading 620
loaded query: alveolar macrophage|Lung|3-F-5... (k=21, DNA)
for alveolar macrophage|Lung|3-F-56|10X_P7_8_ACGGCCACAATGGTCT, found: 0.24 cell_ontology_class:epithelial_cell_of_lung|tissue:Lung|subtissue:nan|free_annotation:alveolar_epithelial_type_1_cells,_alveolar_epithelial_type_2_cells,_club_cells,_and_basal_cells|cell_id:K3-MAA000526-3_9_M-1-1
loaded query: classical monocyte|Lung|3-F-56... (k=21, DNA)
for classical monocyte|Lung|3-F-56|10X_P7_8_ACGAGGACAAGCCCAC, found: 0.40 cell_ontology_class:epithelial_cell_of_lung|tissue:Lung|subtissue:nan|free_annotation:alveolar_epithelial_type_1_cells,_alveolar_epithelial_type_2_cells,_club_cells,_and_basal_cells|cell_id:G4-MAA000526-3_9_M-1-1
loaded query: lung endothelial cell|Lung|3-F... (k=21, DNA)
for lung endothelial cell|Lung|3-F-56|10X_P7_8_AACACGTGTGGCTCCA, found: 0.35 cell_ontology_class:skeletal_muscle_satellite_stem_cell|tissue:Diaphragm|subtissue:nan|free_annotation:nan|cell_id:H3-MAA001454-3_38_F-1-1
loaded query: lung endothelial cell|Lung|3-F... (k=21, DNA)
for lung endothelial cell|Lung|3-F-56|10X_P7_8_CATCGGGCAGATGAGC, found: 0.36 cell_ontology_class:endothelial_cell|tissue:Heart|subtissue:Unknown|free_annotation:nan|cell_id:A2-MAA100037-3_10_M-1-1
loaded query: non-classical monocyte|Lung|3-... (k=21, DNA)
for non-classical monocyte|Lung|3-F-56|10X_P7_8_CTTAACTTCTCAAGTG, found: 0.45 cell_ontology_class:monocyte|tissue:Lung|subtissue:Endomucin|free_annotation:circulating_monocytes|cell_id:A22-MAA001892-3_38_F-1-1
loaded query: stromal cell|Lung|3-F-56|10X_P... (k=21, DNA)
for stromal cell|Lung|3-F-56|10X_P7_8_GTCAAGTAGCTAAACA, found: 0.38 cell_ontology_class:immature_B_cell|tissue:Marrow|subtissue:B-cells|free_annotation:nan|cell_id:E6-MAA001884-3_38_F-1-1
loaded query: stromal cell|Lung|3-F-56|10X_P... (k=21, DNA)
for stromal cell|Lung|3-F-56|10X_P7_8_GTCAAGTAGTTTCCTT, found: 0.27 cell_ontology_class:nan|tissue:Fat|subtissue:SCAT|free_annotation:nan|cell_id:N22-B000127-3_38_F-1-1
loaded query: alveolar macrophage|Lung|3-F-5... (k=21, DNA)
for alveolar macrophage|Lung|3-F-56|10X_P7_8_CCGTACTGTCAGATAA, found: 0.37 cell_ontology_class:monocyte|tissue:Lung|subtissue:Endomucin|free_annotation:circulating_monocytes|cell_id:A22-MAA001892-3_38_F-1-1
loaded query: lung endothelial cell|Lung|3-F... (k=21, DNA)
for lung endothelial cell|Lung|3-F-56|10X_P7_8_CTCAGAAAGTTTCCTT, found: 0.37 cell_ontology_class:skeletal_muscle_satellite_stem_cell|tissue:Diaphragm|subtissue:nan|free_annotation:nan|cell_id:H3-MAA001454-3_38_F-1-1
loaded query: non-classical monocyte|Lung|3-... (k=21, DNA)
for non-classical monocyte|Lung|3-F-56|10X_P7_8_GTCGGGTCACACATGT, found: 0.36 cell_ontology_class:epithelial_cell_of_lung|tissue:Lung|subtissue:nan|free_annotation:alveolar_epithelial_type_1_cells,_alveolar_epithelial_type_2_cells,_club_cells,_and_basal_cells|cell_id:G4-MAA000526-3_9_M-1-1
loaded query: lung endothelial cell|Lung|3-F... (k=21, DNA)
for lung endothelial cell|Lung|3-F-56|10X_P7_8_CCACGGATCAGCTCGG, found: 0.45 cell_ontology_class:bladder_cell|tissue:Bladder|subtissue:nan|free_annotation:Bladder_mesenchymal_cell|cell_id:A3-D041914-3_8_M-1-1
loaded query: non-classical monocyte|Lung|3-... (k=21, DNA)
for non-classical monocyte|Lung|3-F-56|10X_P7_8_GTAACGTAGATAGGAG, found: 0.56 cell_ontology_class:monocyte|tissue:Lung|subtissue:Endomucin|free_annotation:circulating_monocytes|cell_id:A22-MAA001892-3_38_F-1-1
loaded query: alveolar macrophage|Lung|3-F-5... (k=21, DNA)
for alveolar macrophage|Lung|3-F-56|10X_P7_8_GTTTCTAAGTGCAAGC, found: 0.18 cell_ontology_class:epithelial_cell_of_lung|tissue:Lung|subtissue:nan|free_annotation:alveolar_epithelial_type_1_cells,_alveolar_epithelial_type_2_cells,_club_cells,_and_basal_cells|cell_id:G4-MAA000526-3_9_M-1-1
loaded query: B cell|Lung|3-F-56|10X_P7_8_CT... (k=21, DNA)
for B cell|Lung|3-F-56|10X_P7_8_CTGATCCGTTGCGCAC, found: 0.23 cell_ontology_class:immature_B_cell|tissue:Marrow|subtissue:B-cells|free_annotation:nan|cell_id:G8-MAA000652-3_10_M-1-1
loaded query: alveolar macrophage|Lung|3-F-5... (k=21, DNA)
for alveolar macrophage|Lung|3-F-56|10X_P7_8_AACTCTTGTTCCAACA, found: 0.50 cell_ontology_class:monocyte|tissue:Lung|subtissue:Endomucin|free_annotation:circulating_monocytes|cell_id:A22-MAA001892-3_38_F-1-1
loaded query: myeloid cell|Lung|3-F-56|10X_P... (k=21, DNA)
for myeloid cell|Lung|3-F-56|10X_P7_8_GCTGCTTTCATGCTCC, found: 0.22 cell_ontology_class:monocyte|tissue:Lung|subtissue:Endomucin|free_annotation:circulating_monocytes|cell_id:A22-MAA001892-3_38_F-1-1
loaded query: stromal cell|Lung|3-F-56|10X_P... (k=21, DNA)
for stromal cell|Lung|3-F-56|10X_P7_8_AACCATGCACGTCAGC, found: 0.42 cell_ontology_class:endothelial_cell|tissue:Fat|subtissue:BAT|free_annotation:nan|cell_id:F21-B001216-3_38_F-1-1
loaded query: T cell|Lung|3-F-56|10X_P7_8_CC... (k=21, DNA)
for T cell|Lung|3-F-56|10X_P7_8_CCATGTCCACGGTTTA, found: 0.25 cell_ontology_class:T_cell|tissue:Lung|subtissue:EPCAM|free_annotation:nan|cell_id:H1-MAA001889-3_38_F-1-1
loaded query: alveolar macrophage|Lung|3-F-5... (k=21, DNA)
for alveolar macrophage|Lung|3-F-56|10X_P7_8_CGTTGGGCACCGAAAG, found: 0.28 cell_ontology_class:epithelial_cell_of_lung|tissue:Lung|subtissue:nan|free_annotation:alveolar_epithelial_type_1_cells,_alveolar_epithelial_type_2_cells,_club_cells,_and_basal_cells|cell_id:K3-MAA000526-3_9_M-1-1
loaded query: T cell|Lung|3-F-56|10X_P7_8_CC... (k=21, DNA)
for T cell|Lung|3-F-56|10X_P7_8_CCGTTCACATAGTAAG, found: 0.25 cell_ontology_class:epithelial_cell_of_large_intestine|tissue:Large_Intestine|subtissue:Distal|free_annotation:Lgr5+_undifferentiated_cell_(Distal)|cell_id:M2-MAA001873-3_38_F-1-1
loaded query: lung endothelial cell|Lung|3-F... (k=21, DNA)

```

In [1]:
mkdir /mnt/data/10X_P7_8/sourmash_search/


### Can't run this because there's multiple signatures in the sig file

```
sourmash search --ksize 27 --dna --ignore-abundance \
    --output /mnt/data/10X_P7_8/sourmash_search/tabula-muris-dna-k27.csv \
    /mnt/data/10X_P7_8/possorted_genome_bam.sig \
    /mnt/data/maca-facs-sourmash_index_all/tabula-muris-k27-dna/tabula-muris-k27-dna/tabula-muris-k27-dna.sbt.json 
```

In [2]:
mkdir /mnt/data/10X_P7_8/sourmash_categorize

```
sourmash categorize --ksize 27 --dna \
    --csv /mnt/data/10X_P7_8/sourmash_categorize/tabula-muris-dna-k27_ignore-abundance=True.csv \
    --ignore-abundance \
    /mnt/data/maca-facs-sourmash_index_all/tabula-muris-k27-dna/tabula-muris-k27-dna/tabula-muris-k27-dna.sbt.json  \
    /mnt/data/10X_P7_8/possorted_genome_bam.sig
```

In [3]:
ll  /mnt/data/10X_P7_8/sourmash_categorize

total 0
-rw-rw-r-- 1 ubuntu 0 Sep 21 02:15 tabula-muris-dna-k27.csv
-rw-rw-r-- 1 ubuntu 0 Sep 21 02:15 tabula-muris-dna-k27_ignore-abundance=True.csv
